In [27]:
!pip install rdflib
!pip install pandas
from rdflib import *
from hashlib import sha256


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


# Create Source Graph

In [28]:
schema = Graph()
schema.bind('tb', 'http://www.semanticweb.org/acraf/ontologies/2024/healthmesh/tbox#')
schema.bind('ab', 'http://www.semanticweb.org/acraf/ontologies/2024/healthmesh/abox#')
tbox = Namespace('http://www.semanticweb.org/acraf/ontologies/2024/healthmesh/tbox#')
abox = Namespace('http://www.semanticweb.org/acraf/ontologies/2024/healthmesh/abox#')
dcat = Namespace('https://www.w3.org/ns/dcat#')
dcterms = Namespace('http://purl.org/dc/terms/')
dqv = Namespace('http://www.w3.org/ns/dqv#')

## Load Data

In [29]:
import pandas as pd

In [30]:
data_path = '/DataProductLayer/DataProduct/Data/Explotation/UPENN-GBM_clinical_info_v2.1.csv'
df = pd.read_csv(data_path)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 671 entries, 0 to 670
Data columns (total 12 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   ID                                  671 non-null    object 
 1   Gender                              671 non-null    object 
 2   Age_at_scan_years                   671 non-null    float64
 3   Survival_from_surgery_days_UPDATED  671 non-null    object 
 4   Survival_Status                     671 non-null    object 
 5   Survival_Censor                     671 non-null    object 
 6   IDH1                                671 non-null    object 
 7   MGMT                                671 non-null    object 
 8   KPS                                 671 non-null    object 
 9   GTR_over90percent                   671 non-null    object 
 10  Time_since_baseline_preop           671 non-null    object 
 11  PsP_TP_score                        60 non-nu

## Data Product Profile

### OWNER

In [31]:
metadata_path = "/DataProductLayer/DataProduct/Metadata/explotaiton.csv"

In [32]:
meta_df = pd.read_csv(metadata_path)
meta_dp = meta_df.loc[meta_df['name'] == "UPENN-GBM_clinical_info_v2"]
owner = meta_dp.owner.values[0]


### NAME

In [33]:
dataset_name = meta_dp.name.values[0]
dataset_name

'UPENN-GBM_clinical_info_v2'

add triples

In [34]:
# Name
schema.add((abox[dataset_name], RDF.type, dcat.Dataset))
#identifier
identifier = sha256(dataset_name.encode('utf-8')).hexdigest()
schema.add((abox[dataset_name],dcterms.identifier, Literal(identifier)))
#owner
schema.add((abox[dataset_name], tbox.owner, Literal(owner)))


<Graph identifier=N4a264342081f470096bc8d8bfccea21f (<class 'rdflib.graph.Graph'>)>

### ATTRIBUTES


In [35]:
for column in df.columns:
    # Create RDF triple for Attribute
    schema.add((abox[column], RDF.type, tbox.Attribute))
    # Create RDF triple for Attribute
    schema.add((abox[dataset_name], tbox.hasAttribute, abox[column]))
    # rdf for the dataproperty
    schema.add((abox[column], tbox.attribute, Literal(column)))




## DatasetTypeTemplate

### Format

In [36]:
format = meta_dp.format.values[0]
format

'application/csv'

In [37]:
# triple
schema.add((abox.Tabular, RDF.type, tbox.DatasetTypeTemplate))

# format
schema.add((abox.Tabular, dcterms['format'], Literal(format)))  # Correct usage of the namespace

# addDTT
schema.add((abox[dataset_name], tbox.hasDTT, abox.Tabular))


<Graph identifier=N4a264342081f470096bc8d8bfccea21f (<class 'rdflib.graph.Graph'>)>

### TechnologyAspects

In [38]:
#triple
schema.add((abox.TA1, RDF.type, tbox.TechnologyAspects))

#  addTA
schema.add((abox[dataset_name], tbox.hasTA, abox.TA1))

#TYPE
schema.add((abox.TA1, tbox.typeAcces, abox.Static))
schema.add((abox.Static, RDF.type, tbox.Acces))

# PATH
schema.add((abox.Static, tbox.path, Literal(data_path)))


<Graph identifier=N4a264342081f470096bc8d8bfccea21f (<class 'rdflib.graph.Graph'>)>

# SAVE SOURCE_GRAPH

In [39]:
rdf_format = "turtle"
schema.serialize(destination="./Source_Graph_DP1.ttl", format=rdf_format)

<Graph identifier=N4a264342081f470096bc8d8bfccea21f (<class 'rdflib.graph.Graph'>)>